In [ ]:
!pip install diffusers
!pip install transformers
!pip install scipy
!pip install ftfy
!pip install accelerate
!pip install modin[all]
!pip install gradio

In [ ]:
import torch #needed only for GPU
from PIL import Image
from io import BytesIO
from diffusers import StableDiffusionLatentUpscalePipeline
import gradio as gr
import modin.pandas as pd

# load model for CPU or GPU

model_id = "stabilityai/sd-x2-latent-upscaler"
device = "cuda" if torch.cuda.is_available() else "cpu"
upscaler = StableDiffusionLatentUpscalePipeline.from_pretrained(model_id, torch_dtype=torch.float16, safety_checker=None) if torch.cuda.is_available() else StableDiffusionLatentUpscalePipeline.from_pretrained(model_id, safety_checker=None)
upscaler = upscaler.to(device)

#define interface 

def upscale(low_res_img, prompt, negative_prompt, scale, steps):
    generator = torch.manual_seed(69)
    low_res_img = Image.open(low_res_img).convert("RGB")
    low_res_latents = low_res_img
    upscaled_image = upscaler(prompt=prompt, negative_prompt=negative_prompt, image=low_res_latents, guidance_scale=scale, num_inference_steps=steps).images[0]
    return upscaled_image
    
#launch interface
    
gr.Interface(fn=upscale, inputs=[
    gr.Image(type='filepath', label='Lower Resolution Image (768x768 or less, i.e. 128x128, 256x256, 512x512, ect..)'), 
    gr.Textbox(label="Optional: Enter a Prompt to Slightly Guide the AI's Enhancement"), 
    gr.Textbox(label='Experimental: Slightly influence What you do not want the AI to Enhance.'), 
    gr.Slider(2, 15, 7, step=1, label='Guidance Scale: How much the AI influences the Upscaling.'), 
    gr.Slider(5, 25, 10, step=1, label='Number of Iterations')], 
             outputs=gr.Image(type='filepath'), 
             title='SD 2.0 2x Upscaler', 
             description='A 2x Upscaler using SD 2.0. This version runs on CPU or GPU and is currently running on CPU. 10 Iterations is all that is needed to produce good results in most cases. 10 iterations takes ~7mins for 512x512 and ~14mins for 768x768.<br><br><b>Notice: Largest Accepted Resolution is 768x768', 
             article = "Code Monkey: <a href=\"https://huggingface.co/Manjushri\">Manjushri</a>").launch(max_threads=True, debug=True)